# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [30]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [31]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [51]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
#sfo_data.head()


file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)
#df_neighborhood_locations.head()

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [57]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    df_units_pyr = sfo_data['housing_units'].groupby(sfo_data.index).mean()
    
    fig_units_pyr = plt.figure()
    plot_units_pyr = df_units_pyr.plot.bar()
    plot_units_pyr.set_xlabel("Year", fontsize=12)
    plot_units_pyr.set_ylabel("Housing Units", fontsize=12)
    plot_units_pyr.set_title( "Housing Units in San Francisco")

    units_pyr_std = df_units_pyr.std()
    units_pyr_min = min(df_units_pyr)
    units_pyr_max = max(df_units_pyr)

    #print(units_pyr_std)
    plot_units_pyr.set_ylim(units_pyr_min - units_pyr_std, units_pyr_max + units_pyr_std)
    plt.close(fig_units_pyr)


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    df_avg_cost = (sfo_data[["sale_price_sqr_foot", "gross_rent"]].groupby(sfo_data.index).mean())
    
    fig = plt.figure()
    bar_chart = df_avg_cost['sale_price_sqr_foot'].plot(color='gold')
    bar_chart.set_xlabel('Year', fontsize=12)
    bar_chart.set_ylabel('Avg Price Per SgFt By Year', fontsize=12)
    bar_chart.set_title('Average Price per SqFt by Year', fontsize=14, fontweight="bold")
    plt.close(fig)



    # Line chart for average montly rent
    fig = plt.figure()
    bar_chart = df_avg_cost['gross_rent'].plot(color='black')
    bar_chart.set_xlabel('Year', fontsize=12)
    bar_chart.set_ylabel('Avg Price Per SgFt By Year', fontsize=12)
    bar_chart.set_title('Average Gross Rent by Year', fontsize=14, fontweight="bold")
    plt.close(fig)




def average_sales_price():
    """Average Sales Price Per Year."""
    
    df_costs = sfo_data.groupby([sfo_data.index, "neighborhood"]).mean()
    df_costs.reset_index(inplace=True)
    df_costs.rename(columns={"level_0":"year"},inplace=True)



def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    df_costs = sfo_data.groupby([sfo_data.index, "neighborhood"]).mean()
    df_costs.reset_index(inplace=True)
    df_costs.rename(columns={"level_0":"year"},inplace=True)
    
    df_costs.hvplot.line("year", "sale_price_sqr_foot", 
                    xlabel="year",
                    ylabel="Avg. Sale Price per SqFt",
                   groupby="neighborhood",)
    
    
    df_costs.hvplot.line ("year", "gross_rent",
        xlabel="Year",
        ylabel="Avg. Sales Price Per SqFt",
        groupby="neighborhood",)



def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    df_expensive_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    df_expensive_neighborhoods = df_expensive_neighborhoods.sort_values(
        by="sale_price_sqr_foot", ascending=False).head(10)
    df_expensive_neighborhoods = df_expensive_neighborhoods.reset_index()

    df_expensive_neighborhoods.hvplot.bar("neighborhood", "sale_price_sqr_foot",
                                    title="Top 10 Expesive Neighborhoods in San Fransico",
                                    xlabel="Neighborhood",
                                    ylabel ="Avg. Sales Price per SqFt",
                                    height=400,
                                    rot=90
                                     )

def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    df_expensive_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    df_expensive_neighborhoods = df_expensive_neighborhoods.sort_values(
        by="sale_price_sqr_foot", ascending=False).head(10)
    df_expensive_neighborhoods = df_expensive_neighborhoods.reset_index()
    
    px.parallel_categories(
    df_expensive_neighborhoods, color="sale_price_sqr_foot", 
    color_continuous_scale=px.colors.sequential.Blackbody,
    title="Parallel Catagories Analysis of Most Expensive Neighborhoods")

    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    df_expensive_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    df_expensive_neighborhoods = df_expensive_neighborhoods.sort_values(
        by="sale_price_sqr_foot", ascending=False).head(10)
    df_expensive_neighborhoods = df_expensive_neighborhoods.reset_index()
    
    
    px.parallel_coordinates(df_expensive_neighborhoods,
                        color="sale_price_sqr_foot",
                        color_continuous_scale=px.colors.sequential.Blackbody,
                        title="Parallel Coordinate Analysis of Most Expensive Neighboorhoods")



def parallel_categories():
    """Parallel Categories Plot."""
    
    
    df_expensive_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    df_expensive_neighborhoods = df_expensive_neighborhoods.sort_values(
        by="sale_price_sqr_foot", ascending=False).head(10)
    df_expensive_neighborhoods = df_expensive_neighborhoods.reset_index()
    
    px.parallel_coordinates(df_expensive_neighborhoods,
                        color="sale_price_sqr_foot",
                        color_continuous_scale=px.colors.sequential.Blackbody,
                        title="Parallel Coordinate Analysis of Most Expensive Neighboorhoods")



def neighborhood_map():
    """Neighborhood Map."""

    neighborhood_map = px.scatter_mapbox(
    df_all_neigborhoods,
    lat="Lat",
    lon="Lon",
    size="sale_price_sqr_foot",
    color="gross_rent",
    color_continuous_scale=px.colors.cyclical.IceFire,
    size_max=15,
    zoom=11,
    hover_name="Neighborhood",
    title="Average Sale Price per SqFt & Gross Rent in San Francisco",)

def sunburst():
    """Sunburst Plot."""
    df_expensive_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    df_expensive_neighborhoods = df_expensive_neighborhoods.sort_values(
        by="sale_price_sqr_foot", ascending=False).head(10)
    df_expensive_neighborhoods = df_expensive_neighborhoods.reset_index()
    df_expensive_neighborhoods
    
    fig = px.sunburst(
    df_expensive_neighborhoods_per_year,
    path=["year", "neighborhood"],
    values='sale_price_sqr_foot',
    color='gross_rent',
    color_continuous_scale='blackbody',
    title="Cost Analysis of Most Expensive Neighborhood per Year",
    height=700)


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [63]:
# Create a Title for the Dashboard
title = pn.pane.Markdown("""# Real Estate Analysis of San Francisco from 2010 to 2016""", width=700,
)


# Create a tab layout for the dashboard
tabs = pn.Tabs(
    ("Yearly Market Analysis", pn.Row(housing_units_per_year(), average_gross_rent(), average_sales_price())),
    ("Neighborhood Analysis", pn.Column(average_price_by_neighborhood(), top_most_expensive_neighborhoods(), most_expensive_neighborhoods_rent_sales())),
    ("Parallel Plots Analysis", pn.Column(parallel_coordinates(),parallel_coordinates())),
    #("Sunburst Plot Analysis", pn.Column(sunburst())),)


# Create the dashboard
dashboard = pn.Cloumn(pn.Row(title), tabs, width=800)

SyntaxError: unexpected EOF while parsing (<ipython-input-63-78111ed6dc3a>, line 15)

## Serve the Panel Dashboard

In [59]:
# Serve the# dashboard
dashboard.servable

NameError: name 'dashboard' is not defined

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()